# Init

In [3767]:
import warnings
warnings.filterwarnings('ignore')

In [3768]:
from datetime import date
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter

In [3769]:
from bokeh.io import curdoc
from bokeh.core.enums import SizingMode
from bokeh.layouts import layout, column, row, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS, TextInput, Div, RangeSlider, DateRangeSlider, Select, Styles, Range1d, InlineStyleSheet, FuncTickFormatter, CustomJSTickFormatter, BasicTicker, Label, Arrow, NormalHead, Title
from bokeh.plotting import figure, output_file, save, show

# Helpers

In [3770]:
# Index by year and quarter.
# Helper function for a couple
# of different datasets.
def monthToQuarter(qrt):
    month = ''
    if (qrt == 'Qtr1'):
        month = 'Jan'
    elif (qrt == 'Qtr2'):
        month = 'Apr'
    elif (qrt == 'Qtr3'):
        month = 'Aug'
    elif (qrt == 'Qtr4'):
        month = 'Oct'
    else:
        raise Exception('Cannot convert quarter to month')
    return month

# Adds a percent sign to a number.
def percent_formatter(x):
    return f"{x}%"

## Datasource

In [3771]:
# Use BLS-issued series ID.
CPI_SRC = 'data/CUUR0000AA0.xlsx'
UNEMP_SRC = 'data/LNS14000000.xlsx'
ECI_SRC = 'data/CIU2020000000000A.xlsx'
EMP_SRC = 'data/CES0000000001.xlsx'
PROD_SRC = 'data/PRS85006092.xlsx'

## Settings

In [3772]:
LONG_HISTORY_START_YEAR = 1954  # CPI and unemployment data go back to 1954. Base CPI = 1967
SHORT_HISTORY_START_YEAR = 2001. # Other data only go back to 2001.
END_YEAR = 2024
MEDIUM_TERM = '2001-01-01'  # Dotcom bust. Many charts begin here; leave as-is.
SHORT_TERM = '2008-01-01'  # Great recession.
SHORTER_TERM = '2020-01-01'  # Pandemic.

In [3773]:
DISABLE_PLOT = True

# Datasets

## CPI

### Dataset

In [3774]:
# Load datasource.
df_cpi_orig = pd.read_excel(CPI_SRC, skiprows=list(range(11)))
df_cpi_orig.drop(['HALF1', 'HALF2'], inplace=True, axis=1)

# CPI with 1967 dollars
df_cpi_orig = df_cpi_orig[df_cpi_orig['Year'] >= LONG_HISTORY_START_YEAR]

# Index by date.
df_cpi_melted = df_cpi_orig.melt(id_vars='Year', var_name='Month', value_name='Value')
df_cpi_melted['Date'] = df_cpi_melted['Year'].astype(str) + '-' + df_cpi_melted['Month']
df_cpi_melted['Date'] = pd.to_datetime(df_cpi_melted['Date'], format='%Y-%b')
df_cpi = df_cpi_melted.set_index('Date')
df_cpi = df_cpi.sort_index()

### Matplotlib

In [3775]:
if (DISABLE_PLOT == False):
    plt.yscale('log')
    plt.plot(df_cpi.index, df_cpi['Value'])

In [3776]:
if (DISABLE_PLOT == False):
    plt.yscale('log')
    plt.plot(df_cpi[df_cpi.index >= MEDIUM_TERM].index, df_cpi[df_cpi.index >= MEDIUM_TERM]['Value']) # Other charts start at 2001.

### Bokeh (CPI Level)

In [3777]:
# Plot setup.
cpi_plot = figure(
    height=400,
    width=800,
    title="CPI",
    x_axis_label='Date',
    y_axis_label='1967 dollars (%)',
    x_axis_type="datetime",
    y_axis_type="log",
    tools="",
)

# Date range widget.
cpi_date_range_slider = DateRangeSlider(
    title="Period",
    value=(date(LONG_HISTORY_START_YEAR, 1, 1), date(END_YEAR, 1, 1)), # Hard code for now.
    start=date(LONG_HISTORY_START_YEAR, 1, 1),
    end=date(END_YEAR, 1, 1),
    step=365,
)

# Bind slider to plot for auto updating.
cpi_date_range_slider.js_link('value', cpi_plot.x_range, 'start', attr_selector=0)
cpi_date_range_slider.js_link('value', cpi_plot.x_range, 'end', attr_selector=1)

# Data source.
cpi_source = ColumnDataSource(data={'x': df_cpi.index, 'y': df_cpi['Value'], 'smoothed_y': df_cpi['Value']})

# Line plot.
cpi_line = cpi_plot.line('x', 'y', source=cpi_source, line_width=1, line_alpha=0.6, legend_label='Actual')

# Smoothed line plot.
cpi_smoothed_line = cpi_plot.line('x', 'smoothed_y', source=cpi_source, line_width=2, color='red', legend_label='Smooth')

# Resize viewport widget.
cpi_view_size_select = Select(title="Sizing mode", value="fixed", options=list(SizingMode), width=300)
cpi_view_size_select.js_link('value', cpi_plot, 'sizing_mode')

# Resize window widget.
cpi_window_callback = CustomJS(args=dict(source=cpi_source, smoothed_line=cpi_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = cb_obj.value;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
cpi_window_size_slider = Slider(start=3, end=100, value=5, step=2, title="Average")
cpi_window_size_slider.js_on_change('value', cpi_window_callback)

# Line thickness widget.
cpi_thickness_slider_line = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_thickness_slider_line_callback = CustomJS(args=dict(line=cpi_line, slider=cpi_thickness_slider_line), code="""
    line.glyph.line_width = slider.value;
""")
cpi_thickness_slider_line.js_on_change('value', cpi_thickness_slider_line_callback)

# Smoothed line thickness widget.
cpi_thickness_slider_smoothed = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_thickness_slider_smoothed_callback = CustomJS(args=dict(line=cpi_smoothed_line, slider=cpi_thickness_slider_smoothed), code="""
    line.glyph.line_width = slider.value;
""")
cpi_thickness_slider_smoothed.js_on_change('value', cpi_thickness_slider_smoothed_callback)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
cpi_plot.xaxis.ticker = xticker
cpi_plot.xaxis.ticker.num_minor_ticks = 0
cpi_plot.xaxis.axis_line_width = 2
cpi_plot.xaxis.axis_line_color = "#666666"
cpi_plot.xaxis.axis_label_text_color = "#666666"
cpi_plot.xaxis.axis_label_text_font_size = "9.5pt"
cpi_plot.xaxis.major_label_text_font_size = "9pt"
cpi_plot.xaxis.major_label_text_font_size = "9pt"
cpi_plot.xaxis.major_label_text_font_style = "bold"

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
cpi_plot.yaxis.ticker = yticker
cpi_plot.yaxis.ticker.num_minor_ticks = 0
cpi_plot.yaxis.axis_line_width = 2
cpi_plot.yaxis.axis_line_color = "#666666"
cpi_plot.yaxis.axis_label_text_color = "#666666"
cpi_plot.xaxis.axis_label_text_font_size = "9.5pt"
cpi_plot.yaxis.major_label_text_font_size = "9pt"
cpi_plot.yaxis.major_label_text_font_size = "9pt"
cpi_plot.yaxis.major_label_text_font_style = "bold"
cpi_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

cpi_plot.ygrid.grid_line_color = None

# Build layout.
cpi_layout = column(
    cpi_window_size_slider,
    cpi_date_range_slider,
    cpi_view_size_select,
    cpi_thickness_slider_line,
    cpi_thickness_slider_smoothed,
    cpi_plot
)

### Inflation Rate Dataset

In [3778]:
# Copy df_cpi, add rate column.
df_cpi_rate = df_cpi.copy()
df_cpi_rate['Rate'] = ((df_cpi_rate['Value'] - df_cpi_rate['Value'].shift(1)) / df_cpi_rate['Value'].shift(1)) * 100
df_cpi_rate.loc['1954-01-01', 'Rate'] = df_cpi_rate.loc['1954-02-01', 'Rate']  # Backfill first month

### Bokeh (Inflation Rate)

In [3779]:
# Plot setup.
cpi_rate_plot = figure(
    height=400,
    width=728,
    title="CPI Rate",
    x_axis_label='Date',
    y_axis_label='Rate',
    x_axis_type="datetime",
    y_axis_type="linear",
    tools="",
)

# Title size.
cpi_rate_plot.title.text_font_size = '13pt'
cpi_rate_plot.title.text_color="#000000"
cpi_rate_plot.title.standoff = 45

# Date range widget.
cpi_rate_date_range_slider = DateRangeSlider(
    title="Period",
    value=(date(LONG_HISTORY_START_YEAR, 1, 1), date(END_YEAR, 1, 1)), # Hard code for now.
    start=date(LONG_HISTORY_START_YEAR, 1, 1),
    end=date(END_YEAR, 1, 1),
    step=365,
)

# Bind slider to plot for auto updating.
cpi_rate_date_range_slider.js_link('value', cpi_rate_plot.x_range, 'start', attr_selector=0)
cpi_rate_date_range_slider.js_link('value', cpi_rate_plot.x_range, 'end', attr_selector=1)

# Data source.
cpi_rate_source = ColumnDataSource(data={'x': df_cpi_rate.index, 'y': df_cpi_rate['Rate'], 'smoothed_y': df_cpi_rate['Rate']})

# Line plot.
cpi_rate_line = cpi_rate_plot.line('x', 'y', source=cpi_rate_source, line_width=1, line_alpha=0.2, legend_label='Actual')

# Smoothed line plot.
cpi_rate_smoothed_line = cpi_rate_plot.line('x', 'smoothed_y', source=cpi_rate_source, line_width=1.5, color='red', legend_label='Smooth')

# Resize viewport widget.
cpi_rate_view_size_select = Select(title="Sizing mode", value="fixed", options=list(SizingMode), width=300)
cpi_rate_view_size_select.js_link('value', cpi_rate_plot, 'sizing_mode')

# Smooth line on page load. # smoothing value = 50
cpi_rate_callback_default = CustomJS(args=dict(source=cpi_rate_source, smoothed_line=cpi_rate_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = 50;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
curdoc().js_on_event("document_ready", cpi_rate_callback_default)

# Resize window widget.
cpi_rate_window_callback = CustomJS(args=dict(source=cpi_rate_source, smoothed_line=cpi_rate_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = cb_obj.value;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
cpi_rate_window_size_slider = Slider(start=1, end=100, value=50, step=1, title="Average")
cpi_rate_window_size_slider.js_on_change('value', cpi_rate_window_callback)

# Line thickness widget.
cpi_rate_thickness_slider_line = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_rate_thickness_slider_line_callback = CustomJS(args=dict(line=cpi_rate_line, slider=cpi_rate_thickness_slider_line), code="""
    line.glyph.line_width = slider.value;
""")
cpi_rate_thickness_slider_line.js_on_change('value', cpi_rate_thickness_slider_line_callback)

# Smoothed line thickness widget.
cpi_rate_thickness_slider_smoothed = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_rate_thickness_slider_smoothed_callback = CustomJS(args=dict(line=cpi_rate_smoothed_line, slider=cpi_rate_thickness_slider_smoothed), code="""
    line.glyph.line_width = slider.value;
""")
cpi_rate_thickness_slider_smoothed.js_on_change('value', cpi_rate_thickness_slider_smoothed_callback)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=6)
cpi_rate_plot.xaxis.ticker = xticker
cpi_rate_plot.xaxis.ticker.num_minor_ticks = 0
cpi_rate_plot.xaxis.axis_line_width = 2
cpi_rate_plot.xaxis.axis_line_color = "#666666"
cpi_rate_plot.xaxis.axis_label_text_color = "#666666"
cpi_rate_plot.xaxis.axis_label_text_font_size = "9.5pt"
cpi_rate_plot.xaxis.major_label_text_font_size = "9pt"
cpi_rate_plot.xaxis.major_label_text_font_size = "9pt"
cpi_rate_plot.xaxis.major_label_text_font_style = "bold"

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=5)
cpi_rate_plot.yaxis.ticker = yticker
cpi_rate_plot.yaxis.ticker.num_minor_ticks = 0
cpi_rate_plot.yaxis.axis_line_width = 2
cpi_rate_plot.yaxis.axis_line_color = "#666666"
cpi_rate_plot.yaxis.axis_label_text_color = "#666666"
cpi_rate_plot.xaxis.axis_label_text_font_size = "9.5pt"
cpi_rate_plot.yaxis.major_label_text_font_size = "9pt"
cpi_rate_plot.yaxis.major_label_text_font_size = "9pt"
cpi_rate_plot.yaxis.major_label_text_font_style = "bold"
cpi_rate_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

cpi_rate_plot.xgrid.grid_line_color = None

# Build layout.
cpi_rate_layout = column(
    cpi_rate_window_size_slider,
    cpi_rate_date_range_slider,
    cpi_rate_view_size_select,
    cpi_rate_thickness_slider_line,
    cpi_rate_thickness_slider_smoothed,
    cpi_rate_plot
)

## Unemployment

### Dataset

In [3780]:
# Load datasource.
df_unemp_orig = pd.read_excel(UNEMP_SRC, skiprows=list(range(11)))

# Start at CPI with 1967 dollars.
df_unemp_orig = df_unemp_orig[df_unemp_orig['Year'] >= LONG_HISTORY_START_YEAR]

# Index by date.
df_unemp_melted = df_unemp_orig.melt(id_vars='Year', var_name='Month', value_name='Value')
df_unemp_melted['Date'] = df_unemp_melted['Year'].astype(str) + '-' + df_unemp_melted['Month']
df_unemp_melted['Date'] = pd.to_datetime(df_unemp_melted['Date'], format='%Y-%b')
df_unemp = df_unemp_melted.set_index('Date')
df_unemp = df_unemp.sort_index()

### Plot

In [3781]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_unemp.index, df_unemp['Value'])

In [3782]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_unemp[df_unemp.index >= MEDIUM_TERM].index, df_unemp[df_unemp.index >= MEDIUM_TERM]['Value']) # Other charts start at 2001.

### Bokeh

In [3783]:
# Plot setup.
unemp_plot = figure(
    height=400,
    width=728,
    title="Unemployment",
    x_axis_label='Date',
    y_axis_label='Rate',
    x_axis_type="datetime",
    tools="",
)

# Title size.
unemp_plot.title.text_font_size = '13pt'
unemp_plot.title.text_color="#000000"
unemp_plot.title.standoff = 45

# Date range widget.
unemp_date_range_slider = DateRangeSlider(
    title="Period",
    value=(date(LONG_HISTORY_START_YEAR, 1, 1), date(END_YEAR, 1, 1)), # Hard code for now.
    start=date(LONG_HISTORY_START_YEAR, 1, 1),
    end=date(END_YEAR, 1, 1),
    step=365,
)

# Bind slider to plot for auto updating.
unemp_date_range_slider.js_link('value', unemp_plot.x_range, 'start', attr_selector=0)
unemp_date_range_slider.js_link('value', unemp_plot.x_range, 'end', attr_selector=1)

# Data source.
unemp_source = ColumnDataSource(data={'x': df_unemp.index, 'y': df_unemp['Value'], 'smoothed_y': df_unemp['Value']})

# Line plot.
unemp_line = unemp_plot.line('x', 'y', source=unemp_source, line_width=1, line_alpha=0.2, legend_label='Actual')

# Smoothed line plot.
unemp_smoothed_line = unemp_plot.line('x', 'smoothed_y', source=unemp_source, line_width=1.5, color='red', legend_label='Smooth')

# Resize viewport widget.
unemp_view_size_select = Select(title="Sizing mode", value="fixed", options=list(SizingMode), width=300)
unemp_view_size_select.js_link('value', unemp_plot, 'sizing_mode')

# Smooth line on page load. # smoothing value = 50
unemp_window_callback_default = CustomJS(args=dict(source=unemp_source, smoothed_line=unemp_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = 50;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
curdoc().js_on_event("document_ready", unemp_window_callback_default)

# Resize window widget.
unemp_window_callback = CustomJS(args=dict(source=unemp_source, smoothed_line=unemp_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = cb_obj.value;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
unemp_window_size_slider = Slider(start=1, end=100, value=50, step=10, title="Average")
unemp_window_size_slider.js_on_change('value', unemp_window_callback)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=6)
unemp_plot.xaxis.ticker = xticker
unemp_plot.xaxis.ticker.num_minor_ticks = 0
unemp_plot.xaxis.axis_line_width = 2
unemp_plot.xaxis.axis_line_color = "#666666"
unemp_plot.xaxis.axis_label_text_color = "#666666"
unemp_plot.xaxis.axis_label_text_font_size = "9.5pt"
unemp_plot.xaxis.major_label_text_font_size = "9pt"
unemp_plot.xaxis.major_label_text_font_size = "9pt"
unemp_plot.xaxis.major_label_text_font_style = "bold"

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=5)
unemp_plot.yaxis.ticker = yticker
unemp_plot.yaxis.ticker.num_minor_ticks = 0
unemp_plot.yaxis.axis_line_width = 2
unemp_plot.yaxis.axis_line_color = "#666666"
unemp_plot.yaxis.axis_label_text_color = "#666666"
unemp_plot.xaxis.axis_label_text_font_size = "9.5pt"
unemp_plot.yaxis.major_label_text_font_size = "9pt"
unemp_plot.yaxis.major_label_text_font_size = "9pt"
unemp_plot.yaxis.major_label_text_font_style = "bold"
unemp_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

unemp_plot.xgrid.grid_line_color = None

# Build layout.
unemp_layout = column(
    unemp_window_size_slider,
    unemp_date_range_slider,
    unemp_view_size_select,
    unemp_plot
)

## Employment Cost Index

Wages and salaries for Private industry workers in All industries and occupations, 12-month percent change

In [3784]:
# Load datasource
df_eci_orig = pd.read_excel(ECI_SRC, skiprows=list(range(15)))

# Insert month from biz quarter.
df_eci_orig['Month'] = df_eci_orig['Period'].apply(monthToQuarter)

# Make date index
df_eci_orig['Date'] = df_eci_orig.apply(lambda row: f"{row['Year']}-{row['Month']}", axis=1)
df_eci_orig['Date'] = pd.to_datetime(df_eci_orig['Date'], format='%Y-%b')
df_eci = df_eci_orig.set_index('Date')
df_eci = df_eci.sort_index()
df_eci.drop(['Standard Error', 'Month', 'Period', 'Year'], axis=1, inplace=True)

### Plot

In [3785]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_eci.index, df_eci['Estimate Value'])

In [3786]:
# window size, polynomial order 
cost_smooth_y = savgol_filter(df_eci['Estimate Value'], 15, 2)

In [3787]:
if (DISABLE_PLOT == False):
    plt.plot(df_eci.index, cost_smooth_y)
    plt.xlabel("Period")
    plt.ylabel("Change")
    plt.show()

In [3788]:
# window size, polynomial order 
unemp_smooth_y = savgol_filter(df_unemp[df_unemp.index >= '2001-01-01']['Value'], 15, 2)

In [3789]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_unemp[df_unemp.index >= '2001-01-01'].index, unemp_smooth_y) # Other charts start at 2001.

In [3790]:
df_unemp_eci = pd.DataFrame()
df_unemp_eci = pd.merge(df_unemp, df_eci, left_index=True, right_index=True)

In [3791]:
# df_unemp_eci.head(100)

In [3792]:
# window size, polynomial order 
unemp_eci_eci_smooth_y = savgol_filter(df_unemp_eci['Estimate Value'], 15, 2)
unemp_eci_unemp_smooth_y = savgol_filter(df_unemp_eci['Value'], 15, 2)

In [3793]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(unemp_eci_unemp_smooth_y, unemp_eci_eci_smooth_y)

#### Bokeh

In [3794]:
# Plot setup.
uemp_eci_plot = figure(
    title="Unemployment & Compensation",
    height=400,
    width=400,
    x_axis_label='Unemployment',
    y_axis_label='Compensation',
    y_axis_type="linear",
    tools="",
)

# Title size.
uemp_eci_plot.title.text_font_size = '13pt'
uemp_eci_plot.title.text_color="#000000"
uemp_eci_plot.title.standoff = 45

# Unemployment/Compensation level.
uemp_eci_line = uemp_eci_plot.line(unemp_eci_unemp_smooth_y, unemp_eci_eci_smooth_y, line_width=2, color="red", line_alpha=1)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[uemp_eci_line], tooltips=[("Unemployment", "@x"), ("Compensation", "@y{0.0}%")])
uemp_eci_plot.add_tools(hover_tool)

# Annotate
annotation_start = Label(
    x=5,
    y=5,
    text="From Jan 2001...", 
    text_font_size="9pt",
    text_color="black",
    background_fill_color="white",
    background_fill_alpha=0.7
)
annotation_end = Label(
    x=3,
    y=1.75,
    text="...to Oct 2023", 
    text_font_size="9pt",
    text_color="black",
    background_fill_color="white",
    background_fill_alpha=0.7
)
arrow_start = Arrow(
    end=NormalHead(size=10),
    x_start=5, 
    y_start=5,
    x_end=4.6,
    y_end=4.4,
    line_color="black"
)
arrow_end = Arrow(
    end=NormalHead(size=10),
    x_start=3.5, 
    y_start=2,
    x_end=4.2,
    y_end=3.8,
    line_color="black"
)
uemp_eci_plot.add_layout(annotation_start)
uemp_eci_plot.add_layout(annotation_end)
uemp_eci_plot.add_layout(arrow_start)
uemp_eci_plot.add_layout(arrow_end)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
uemp_eci_plot.xaxis.ticker = xticker
uemp_eci_plot.xaxis.ticker.num_minor_ticks = 0
uemp_eci_plot.xaxis.axis_line_width = 2
uemp_eci_plot.xaxis.axis_line_color = "#666666"
uemp_eci_plot.xaxis.axis_label_text_color = "#666666"
uemp_eci_plot.xaxis.axis_label_text_font_size = "9.5pt"
uemp_eci_plot.xaxis.major_label_text_font_size = "9pt"
uemp_eci_plot.xaxis.major_label_text_font_size = "9pt"
uemp_eci_plot.xaxis.major_label_text_font_style = "bold"
uemp_eci_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
uemp_eci_plot.yaxis.ticker = yticker
uemp_eci_plot.yaxis.ticker.num_minor_ticks = 0
uemp_eci_plot.yaxis.axis_line_width = 2
uemp_eci_plot.yaxis.axis_line_color = "#666666"
uemp_eci_plot.yaxis.axis_label_text_color = "#666666"
uemp_eci_plot.xaxis.axis_label_text_font_size = "9.5pt"
uemp_eci_plot.yaxis.major_label_text_font_size = "9pt"
uemp_eci_plot.yaxis.major_label_text_font_size = "9pt"
uemp_eci_plot.yaxis.major_label_text_font_style = "bold"
uemp_eci_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
uemp_eci_layout = column(
    uemp_eci_plot,
)

## Productivity

In [3795]:
# Load dataset.
df_prod_orig = pd.read_excel(PROD_SRC, skiprows=list(range(10)))

# Start at CPI at 1967 dollars.
df_prod_orig = df_prod_orig[df_prod_orig['Year'] >= LONG_HISTORY_START_YEAR]

# Index by date.
df_prod_melted = df_prod_orig.melt(id_vars='Year', var_name='Quarter', value_name='Value')
df_prod_melted['Month'] = df_prod_melted['Quarter'].apply(monthToQuarter)
df_prod_melted['Date'] = df_prod_melted['Year'].astype(str) + '-' + df_prod_melted['Month']
df_prod_melted['Date'] = pd.to_datetime(df_prod_melted['Date'], format='%Y-%b')
df_prod = df_prod_melted.set_index('Date')
df_prod = df_prod.sort_index()
df_prod.drop(['Year', 'Quarter', 'Month'], axis=1, inplace=True)

In [3796]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_prod.index, df_prod['Value'])

In [3797]:
# window size, polynomial order 
prod_smooth_y = savgol_filter(df_prod[df_prod.index >= '2001-01-01']['Value'], 15, 2)

In [3798]:
if (DISABLE_PLOT == False):
    plt.plot(df_prod[df_prod.index >= '2001-01-01'].index, prod_smooth_y)
    plt.xlabel("Period")
    plt.ylabel("Productivity")
    plt.show()

In [3799]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_prod[df_prod.index >= MEDIUM_TERM].index, df_prod[df_prod.index >= MEDIUM_TERM]['Value'])

### Productivity & Employee Cost (Bokeh)

In [3800]:
# Plot setup.
prod_cost_plot = figure(
    title="Productivity & Compensation",
    height=400,
    width=760,
    x_axis_type="datetime",
    x_axis_label='Period',
    y_axis_label='Change',
    y_axis_type="linear",
    tools="",
)

# Title size.
prod_cost_plot.title.text_font_size = '13pt'
prod_cost_plot.title.text_color="#000000"
prod_cost_plot.title.standoff = 3
subtitle = Title(text="Shaded regions indicate matching trends.", text_font_size="9pt", text_font_style="normal")
prod_cost_plot.add_layout(subtitle, 'above')

# Productivity level.
prod_line = prod_cost_plot.line(df_eci.index, prod_smooth_y, legend_label="Productivity", line_width=2, color="red", line_alpha=0.5, hover_alpha=1)

# Employee cost index.
cost_line = prod_cost_plot.line(df_eci.index, cost_smooth_y, legend_label="Compensation", line_width=2, color="blue", line_alpha=0.5, hover_alpha=1)

# Increase alpha on hover.
# hover_tool = HoverTool(renderers=[prod_line, cost_line], tooltips=[("Period", "@x"), ("Rate of change", "@y{0.0}%")], formatters={"@x": "datetime"})
hover_tool = HoverTool(renderers=[prod_line, cost_line], tooltips=[("Period", "@x{%F}"), ("Rate of change", "@y{0.0}%")], formatters={"@x": "datetime"})
prod_cost_plot.add_tools(hover_tool)

# 0000000000000 unix timestamp format

# Downtrend
prod_cost_plot.quad(
    top=[max(prod_smooth_y) + 0.5],
    bottom=[min(prod_smooth_y) - 0.5],
    left=[1033430400000],
    right=[1112313600000],
    color="#000000",
    alpha=0.1,
)

# Uptrend
prod_cost_plot.quad(
    top=[max(prod_smooth_y) + 0.5],
    bottom=[min(prod_smooth_y) - 0.5],
    left=[1325376000000],
    right=[1406851200000],
    color="#000000",
    alpha=0.1,
)

# Uptrend
prod_cost_plot.quad(
    top=[max(prod_smooth_y) + 0.5],
    bottom=[min(prod_smooth_y) - 0.5],
    left=[1451606400000],
    right=[1522540800000],
    color="#000000",
    alpha=0.1,
)

# Annotate
annotation_1 = Label(
    x=1038930400000,
    y=min(prod_smooth_y) + 0.5,
    text="Downtrend", 
    text_font_size="9pt",
    text_color="black",
    # background_fill_color="white",
    # background_fill_alpha=0.7
)

# Annotate
annotation_2 = Label(
    x=1338238400000,
    y=min(prod_smooth_y) + 0.5,
    text="Uptrend", 
    text_font_size="9pt",
    text_color="black",
    # background_fill_color="white",
    # background_fill_alpha=0.7
)

# Annotate
annotation_3 = Label(
    x=1461606400000,
    y=min(prod_smooth_y) + 0.5,
    text="Uptrend", 
    text_font_size="9pt",
    text_color="black",
    # background_fill_color="white",
    # background_fill_alpha=0.7
)

annotation_4 = Label(
    x=950486209000,
    y=max(prod_smooth_y) - 0.5,
    text="2001 Recession", 
    text_font_size="9pt",
    text_color="black",
    # background_fill_color="white",
    # background_fill_alpha=0.7
)
arrow_4 = Arrow(
    end=NormalHead(size=10),
    x_start=983486209000, 
    y_start=max(prod_smooth_y) - 0.5,
    x_end=983486209000,
    y_end=3.9,
    line_color="black"
)

annotation_5 = Label(
    x=1139548609000,
    y=max(prod_smooth_y) - 0.5,
    text="Great Recession", 
    text_font_size="9pt",
    text_color="black",
    # background_fill_color="white",
    # background_fill_alpha=0.7
)
arrow_5 = Arrow(
    end=NormalHead(size=10),
    x_start=1196548609000, 
    y_start=max(prod_smooth_y) - 0.5,
    x_end=1196548609000,
    y_end=3.6,
    line_color="black"
)

annotation_6 = Label(
    x=1503962609000,
    y=max(prod_smooth_y) - 0.5,
    text="Pandemic", 
    text_font_size="9pt",
    text_color="black",
    # background_fill_color="white",
    # background_fill_alpha=0.7
)
arrow_6 = Arrow(
    end=NormalHead(size=10),
    x_start=1553962609000, 
    y_start=max(prod_smooth_y) - 0.5,
    x_end=1570962609000,
    y_end=5.6,
    line_color="black"
)

prod_cost_plot.add_layout(annotation_1)
prod_cost_plot.add_layout(annotation_2)
prod_cost_plot.add_layout(annotation_3)
prod_cost_plot.add_layout(annotation_4)
prod_cost_plot.add_layout(annotation_5)
prod_cost_plot.add_layout(annotation_6)

prod_cost_plot.add_layout(arrow_4)
prod_cost_plot.add_layout(arrow_5)
prod_cost_plot.add_layout(arrow_6)

prod_cost_plot.y_range.start = min(prod_smooth_y) - 0.5
prod_cost_plot.y_range.end = max(prod_smooth_y) + 0.5

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
prod_cost_plot.xaxis.ticker = xticker
prod_cost_plot.xaxis.ticker.num_minor_ticks = 0
prod_cost_plot.xaxis.axis_line_width = 2
prod_cost_plot.xaxis.axis_line_color = "#666666"
prod_cost_plot.xaxis.axis_label_text_color = "#666666"
prod_cost_plot.xaxis.axis_label_text_font_size = "9.5pt"
prod_cost_plot.xaxis.major_label_text_font_size = "9pt"
prod_cost_plot.xaxis.major_label_text_font_size = "9pt"
prod_cost_plot.xaxis.major_label_text_font_style = "bold"

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
prod_cost_plot.yaxis.ticker = yticker
prod_cost_plot.yaxis.ticker.num_minor_ticks = 0
prod_cost_plot.yaxis.axis_line_width = 2
prod_cost_plot.yaxis.axis_line_color = "#666666"
prod_cost_plot.yaxis.axis_label_text_color = "#666666"
prod_cost_plot.xaxis.axis_label_text_font_size = "9.5pt"
prod_cost_plot.yaxis.major_label_text_font_size = "9pt"
prod_cost_plot.yaxis.major_label_text_font_size = "9pt"
prod_cost_plot.yaxis.major_label_text_font_style = "bold"
prod_cost_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

prod_cost_plot.xgrid.grid_line_color = None

# Build layout.
prod_cost_layout = column(
    prod_cost_plot
)

### Unemployment & Employee Cost

In [3801]:
if (DISABLE_PLOT == False):
    plt.plot(df_prod[df_prod.index >= '2001-01-01'].index, prod_smooth_y)
    plt.xlabel("Period")
    plt.ylabel("Productivity")
    plt.show()

In [3802]:
if (DISABLE_PLOT == False):
    plt.plot(df_prod[df_prod.index >= '2001-01-01'].index, prod_smooth_y)
    plt.xlabel("Period")
    plt.ylabel("Productivity")
    plt.show()

## Phillips – Illustration

### Sample dataset

In [3803]:
# Illustration df.
df_phillips_illu = pd.DataFrame({
    'inflation': [15, 8, 7, 4.25, 3, 2.25, 1.625, 1.25, 1, 0.875, 0.625, 0.6, 0.6], 
    'unemployment': [3, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 6.375, 7.5, 8, 14]
})
df_phillips_illu.set_index('unemployment', inplace=True)

# Draws the phillips curve.
def polynomial_func(x, a, b, c):
    return ((a + b) / (x + c * (x**3)))

# Assign x, y data.
x_data = df_phillips_illu.index
y_data = df_phillips_illu['inflation'].values

# Fit a curved line.
x_curve = np.linspace(min(x_data), max(x_data), 100)
y_curve = polynomial_func(x_curve, max(y_data), 100, .6)

### Matplotlib

In [3804]:
if (DISABLE_PLOT == False):
    # Axes.
    plt.xlim(3, 8)
    plt.ylim(0, 8) 
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    
    # Plot curves.
    plt.plot(x_curve, y_curve, color='red', label='Best-fit Curve')
    plt.plot(x_curve, y_curve + 1, color='blue', label='Best-fit Curve')
    plt.plot(x_curve, y_curve + 2, color='green', label='Best-fit Curve')
    
    # Show
    plt.show()

### Bokeh

In [3805]:
# Plot setup.
phillips_illu_plot = figure(
    height=400,
    width=400,
    title="Phillips Curve",
    x_axis_label='Unemployment Rate',
    y_axis_label='Inflation Rate',
    y_axis_type="linear",
    tools="",
)

# Title size.
phillips_illu_plot.title.text_font_size = '13pt'
phillips_illu_plot.title.text_color="#000000"
phillips_illu_plot.title.standoff = 45

# Data source.
phillips_source_1 = ColumnDataSource(data={'x': x_curve, 'y': y_curve})
phillips_source_2 = ColumnDataSource(data={'x': x_curve, 'y': y_curve + 1})
phillips_source_3 = ColumnDataSource(data={'x': x_curve, 'y': y_curve + 2})

# Line plot.
phillips_line_1 = phillips_illu_plot.line('x', 'y', source=phillips_source_1, line_width=2, line_alpha=0.5, line_color="red", legend_label='Tight money', hover_alpha=1)
phillips_line_2 = phillips_illu_plot.line('x', 'y', source=phillips_source_2, line_width=2, line_alpha=0.5, line_color="purple", legend_label='Neutral money', hover_alpha=1)
phillips_line_3 = phillips_illu_plot.line('x', 'y', source=phillips_source_3, line_width=2, line_alpha=0.5, line_color="blue", legend_label='Easy money', hover_alpha=1)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_line_1, phillips_line_2, phillips_line_3], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], line_policy='next')
phillips_illu_plot.add_tools(hover_tool)

# Set x-axis scale.
phillips_illu_plot.x_range.start = 3
phillips_illu_plot.x_range.end = 8

# Set y-axis scale.
phillips_illu_plot.y_range.start = 0
phillips_illu_plot.y_range.end = 8

# X tick intervals
xticker = BasicTicker(desired_num_ticks=4)
phillips_illu_plot.xaxis.ticker = xticker
phillips_illu_plot.xaxis.ticker.num_minor_ticks = 0
phillips_illu_plot.xaxis.axis_line_width = 2
phillips_illu_plot.xaxis.axis_line_color = "#666666"
phillips_illu_plot.xaxis.axis_label_text_color = "#666666"
phillips_illu_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_illu_plot.xaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.xaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.xaxis.major_label_text_font_style = "bold"
phillips_illu_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
phillips_illu_plot.yaxis.ticker = yticker
phillips_illu_plot.yaxis.ticker.num_minor_ticks = 0
phillips_illu_plot.yaxis.axis_line_width = 2
phillips_illu_plot.yaxis.axis_line_color = "#666666"
phillips_illu_plot.yaxis.axis_label_text_color = "#666666"
phillips_illu_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_illu_plot.yaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.yaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.yaxis.major_label_text_font_style = "bold"
phillips_illu_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
phillips_illu_layout = column(
    phillips_illu_plot
)

## Phillips – Scatters

### Dataset

In [3806]:
# Phillips dataframe.
df_phillips = pd.DataFrame()

# Combine inflation rate and unemployment rate.
df_phillips['inflation_rate'] = df_cpi_rate['Rate'] * 10
df_phillips['unemployment_rate'] = df_unemp['Value']

# Clean up dataframe.
df_phillips.dropna(inplace=True)
df_phillips['Date'] = df_phillips.index
df_phillips.set_index('unemployment_rate', inplace=True)

### Philips - 1954-1970

In [3807]:
# Limit year.
df_phillips_1 = df_phillips[df_phillips['Date'] < '1970-01-01']

#### Matplotlib

In [3808]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips_1.index, df_phillips_1['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips_1.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [3809]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1954-1970",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Title size.
phillips_plot.title.text_font_size = '13pt'
phillips_plot.title.text_color="#000000"
phillips_plot.title.standoff = 30

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_1.index, 'y': df_phillips_1['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, size=6, color="blue", alpha=0.5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], mode='mouse')
phillips_plot.add_tools(hover_tool)

# Shaded area.
phillips_plot.patches([[3.4, 6, 3.4]], [[3.4, 3.4, 8]], color=["#000000"], alpha=[0.1], line_width=0)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
phillips_plot.xaxis.ticker = xticker
phillips_plot.xaxis.ticker.num_minor_ticks = 0
phillips_plot.xaxis.axis_line_width = 2
phillips_plot.xaxis.axis_line_color = "#666666"
phillips_plot.xaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_style = "bold"
phillips_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
phillips_plot.yaxis.ticker = yticker
phillips_plot.yaxis.ticker.num_minor_ticks = 0
phillips_plot.yaxis.axis_line_width = 2
phillips_plot.yaxis.axis_line_color = "#666666"
phillips_plot.yaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_style = "bold"
phillips_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
phillips_layout_01 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

### Phillips - 1970-1986

In [3810]:
# Limit year.
df_phillips_2 = df_phillips[(df_phillips['Date'] >= '1970-01-01') & (df_phillips['Date'] < '1986-01-01')]

#### Matplotlib

In [3811]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips_2.index, df_phillips_2['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips_2.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [3812]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1970-1986",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Title size.
phillips_plot.title.text_font_size = '13pt'
phillips_plot.title.text_color="#000000"
phillips_plot.title.standoff = 30

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_2.index, 'y': df_phillips_2['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, color="blue", size=6, alpha=0.5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], mode='mouse')
phillips_plot.add_tools(hover_tool)

# Shaded area.
phillips_plot.patches([[4, 10, 6.25]], [[3.4, 3.4, 14.5]], color=["#000000"], alpha=[0.1], line_width=0)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
phillips_plot.xaxis.ticker = xticker
phillips_plot.xaxis.ticker.num_minor_ticks = 0
phillips_plot.xaxis.axis_line_width = 2
phillips_plot.xaxis.axis_line_color = "#666666"
phillips_plot.xaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_style = "bold"
phillips_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
phillips_plot.yaxis.ticker = yticker
phillips_plot.yaxis.ticker.num_minor_ticks = 0
phillips_plot.yaxis.axis_line_width = 2
phillips_plot.yaxis.axis_line_color = "#666666"
phillips_plot.yaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_style = "bold"
phillips_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
phillips_layout_02 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

### Phillips - 1986-2000

In [3813]:
# Limit year.
df_phillips_3 = df_phillips[(df_phillips['Date'] >= '1986-01-01') & (df_phillips['Date'] < '2000-01-01')]

#### Matplotlib

In [3814]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips_3.index, df_phillips_3['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips_3.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [3815]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1986-2000",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Title size.
phillips_plot.title.text_font_size = '13pt'
phillips_plot.title.text_color="#000000"
phillips_plot.title.standoff = 30

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_3.index, 'y': df_phillips_3['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, color="blue", size=6, alpha=0.5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], mode='mouse')
phillips_plot.add_tools(hover_tool)

# Shaded area.
phillips_plot.patches([[5, 8, 5]], [[3.4, 3.4, 7]], color=["#000000"], alpha=[0.1], line_width=0)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
phillips_plot.xaxis.ticker = xticker
phillips_plot.xaxis.ticker.num_minor_ticks = 0
phillips_plot.xaxis.axis_line_width = 2
phillips_plot.xaxis.axis_line_color = "#666666"
phillips_plot.xaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_style = "bold"
phillips_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
phillips_plot.yaxis.ticker = yticker
phillips_plot.yaxis.ticker.num_minor_ticks = 0
phillips_plot.yaxis.axis_line_width = 2
phillips_plot.yaxis.axis_line_color = "#666666"
phillips_plot.yaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_style = "bold"
phillips_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
phillips_layout_03 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

### Phillips - 1996-2013

In [3816]:
# Limit year.
df_phillips_4 = df_phillips[(df_phillips['Date'] >= '1996-01-01') & (df_phillips['Date'] < '2013-01-01')]

#### Matplotlib

In [3817]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips.index, df_phillips['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [3818]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1996-2013",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Title size.
phillips_plot.title.text_font_size = '13pt'
phillips_plot.title.text_color="#000000"
phillips_plot.title.standoff = 30

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_4.index, 'y': df_phillips_4['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, color="blue", size=6, alpha=0.5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], mode='mouse')
phillips_plot.add_tools(hover_tool)

# Shaded areas.
phillips_plot.patches([[3.4, 6.5, 5]], [[3.4, 3.4, 12]], color=["#000000"], alpha=[0.1], line_width=0)
phillips_plot.patches([[7.5, 10, 9]], [[3.4, 3.4, 12]], color=["#000000"], alpha=[0.1], line_width=0)

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
phillips_plot.xaxis.ticker = xticker
phillips_plot.xaxis.ticker.num_minor_ticks = 0
phillips_plot.xaxis.axis_line_width = 2
phillips_plot.xaxis.axis_line_color = "#666666"
phillips_plot.xaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_size = "9pt"
phillips_plot.xaxis.major_label_text_font_style = "bold"
phillips_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
phillips_plot.yaxis.ticker = yticker
phillips_plot.yaxis.ticker.num_minor_ticks = 0
phillips_plot.yaxis.axis_line_width = 2
phillips_plot.yaxis.axis_line_color = "#666666"
phillips_plot.yaxis.axis_label_text_color = "#666666"
phillips_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_size = "9pt"
phillips_plot.yaxis.major_label_text_font_style = "bold"
phillips_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
phillips_layout_04 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

# Layout

In [3819]:
cpi_unempl_layout = column(
    row(
        column(
            Div(
                text="""
                    <h1>What about the Phillips Curve?</h1>
                    <h2>A once well-established idea is being questioned.</h2>
                    <p><big>Many economists have claimed over the past 50 years that there 
                    exists a substantial inverse relationship between inflation and 
                    unemployment. The core argument claims that as prices rise across 
                    a broad range of goods and services, that employment levels 
                    increase exponentially when inflation goes asymptomatic.<sup>1</sup> This 
                    claim has been increasingly challenged.</big></p>
                    """,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            Div(
                text="""
                    <h2>Origins of the curve.</h2>
                    <p>Economist William Phillips developed this inverse-relationship concept 
                    in the late 1960s, taking cues from earlier economists such as Irving 
                    Fisher, who in the 1920s published work related to inversely related price 
                    levels and employment. Phillips took a different perspective by focusing 
                    on the supply of money, which is in large part determined by operations 
                    performed by the Federal Reserve’s Open Market Committee.<sup>3</sup></p>
                    <h2>The Phillips Curve</h2>
                    <p>By the 1970s, economists formalized Phillips' idea as a rough 
                    mathematical relating the two variables of inflation and unemployment;
                    Phillips' name was attached to the visualizatoin of their relationship,
                    calling it the "Phillips Curve." After 50 years of new economic experiences,
                    his curve is being questioned.<sup>2</sup></p>
                    """,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            width=820,
            styles=Styles(
                padding='30px 30px 30px 60px',   
            )
        ),
        column(
            Div(
                text="""
                <img src="phillip_portrait.png" />
                <p>William Phillips, originator of the "Phillips Curve." <em>Credit: <a href="https://en.wikipedia.org/wiki/Bill_Phillips_(economist)" target="_blank">Wikipedia</a></em></p>
                """,
                stylesheets=[InlineStyleSheet(css="img { max-width: 100%; height: auto; padding: 15px; border: 1px solid #ccc; margin-top: 60px; }")],
            ),
            width=500,
            styles=Styles(
                padding='0 60px 0 30px',
                margin_top="auto",
                margin_bottom="auto",
            ),
        ),
    ),
    row(
        column(
            prod_cost_layout,
            styles=Styles(
                margin='30px 60px 30px 30px',
            ),
        ),
        column(
            Div(
                text="""
                    <h2>Comparing productivity and employment.</h2>
                    <p>Relationships between employment and money are longstanding, and
                    a particularly clear notion comes from comparing labor productivity
                    growth and employment growth.<sup>4</sup></p>
                    <p>Two possible interpretations emerge from the chart on the left. One 
                    could claim a lagging, but direct relationship exists. Another could 
                    claim there's generally an inverse relationship.</p>
                    """,
                width=400,
                styles=Styles(
                    padding='0 30px 0 0',
                ),
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            styles=Styles(
                margin='auto 0',
            ),
        )
    ),
    row(
        column(
            Div(
                text="""
                    <h2>A more relatable correlation?</h2>
                    <p>Comparing productivity and employment growth is interesting, but
                    much more often we want to compare inflation and employment, which is
                    where the Phillips Curve takes its cue. Not only are the two variables
                    relatable by most interested people, but they play an important role in 
                    setting economic expectations more generally.<sup>5</sup></p>
                    <p>The three curves to the right shows the inverse relationship between
                    the rate of inflation (price levels) and the rate of unemployment. Each
                    curve represents the same phenomenon, but with differing quantities of
                    cash circulation, depending on monetary policy decided by the Federal Reserve.<sup>6</sup></p>
                    <h2>The Supply of Money</h2>
                    <p>Generally, the Federal Reserve enacts policies that attempt to expand, 
                    retract, or sustain the amount of money in circulation. Expansionary policies
                    tend to create inflation without increasing production (thus employmen), which
                    would push the curve upwards. "Tight" money during retractions lowers the
                    curve, and "easy" money during expansions raises the curve.<sup>7</sup></p>
                    """,
                width=680,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            styles=Styles(
                margin='30px 60px 30px 75px',
            )
        ),
        column(
            phillips_illu_layout,
            styles=Styles(
                margin='auto 0',
            ),
        )
    ),
    row(
        column(
            phillips_layout_01,
        ),
        column(
            phillips_layout_02,
        ),
        column(
            Div(
                text="""
                    <h2>Noisy Patterns</h2>
                    <p>Although a correlation predicted by the Phillips Curve may be roughly
                    sussed out in the 1954-1970 scatterplot, the relationship decidedly 
                    breaks apart in the following period. The gray shade shows a very rough
                    trend—or lack of one.</p>
                    """,
                width=400,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            styles=Styles(
                padding='30px 30px 30px 60px',
                margin_top="auto",
                margin_bottom="auto",
            ),
        ),
        styles=Styles(
            padding='30px',
        ),
    ),
    row(
        column(
            phillips_layout_03,
        ),
        column(
            phillips_layout_04,
        ),
        column(
            Div(
                text="""
                    <h2>Very Inconsistent</h2>
                    <p>Once again the relationship claimed by Phillips seems to partly
                    re-appear between 1986 and 2000, but after the Dotcom crash and the
                    following set of recessions break up the chart once again, and in 
                    a very intersesting, somewhat bifurcated fashion.</p>
                    """,
                width=400,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            styles=Styles(
                padding='30px 30px 30px 60px',
                margin_top="auto",
                margin_bottom="auto",
            ),
        ),
        styles=Styles(
            padding='0 30px 30px',
        ),
    ),
    row(
        column(
            Div(
                text="""
                    <h2>Any other correlations?</h2>
                    <p>Another commonly cited relationship is between employment and 
                    compensation. Studies of their possible correlations date alongside
                    the development of the Phillips Curve, which swaps the inflation
                    variable with earned wages, salaries, and benefits.</p>
                    <p>The chart at the right shows a multi-year "walk" of this relationship,
                    tracking each quarter from January 2001 to October 2023. Here, a very
                    clear inverse relationship appears, which would make sense to most
                    anyone who's seeked and help employment through economic cycles.</p>
                    <h2>William Phillips tested other ideas.</h2>
                    <p>As an economist, Phillips was skeptical of his own work and followed
                    many threads of theory, and employment vs. compensation was no different.
                    Around the same time he was working on his namesake idea, he published 
                    <a href="https://en.wikipedia.org/wiki/Phillips_curve#/media/File:Phillips_Curve.svg" target="_blank">this
                    wage vs. unemployment plot</a> that seems awfully similar to the chart at right.</p>
                    """,
                width=680,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            styles=Styles(
                margin='30px 60px 30px 75px',
            ),
        ),
        column(
            uemp_eci_layout,
        ),
    ),
    row(
        column(
            Div(
                text="""
                    <h1>Go ahead, take a look!</h1>
                    <p><big>Play with the two charts below and see if you can find a pattern.</big></p>
                    <p>Maybe you can detect some sort of relationship between inflation and
                    employment. Data provided by the Bureau of Labor Statistics and reports
                    from the media often show noisy short-term patterns, or overly-broad
                    undetailed visualizations. But there are many ways to remove noise
                    and hone in on certain periods in economic history to see if, or when,
                    a correlation could manifest.</p>
                    <p>Play with and compare the two charts below by smoothing the published
                    data and adjusting the date periods. Do you notice a Phillips-like 
                    phenomenon around the 1970s? If so, do you wonder why that phenomenon
                    seems to have vanished in recent decades?</p>
                    """,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            styles=Styles(
                padding='60px 80px 0',
                margin_top='60px',
                margin_left="80px",
                margin_right="80px",
                border='1px solid #ccc',
                border_bottom='none',
                border_top_left_radius="30px",
                border_top_right_radius="30px",
                # background='#f9f9f9',
            ),
            width=1160,
        ),
    ),
    row(
        column(
            cpi_rate_plot,
        ),
        column(
            cpi_rate_window_size_slider,
            cpi_rate_date_range_slider,
            styles=Styles(
                padding='30px 30px 0',
                margin_top='auto',
                margin_bottom='auto',
            ),
        ),
        styles=Styles(
            padding='30px 30px 0',
            border_left="1px solid #ccc",
            margin_left="80px",
            border_right="1px solid #ccc",
            margin_right="80px",
            # background_color="#f9f9f9",
        ),
    ),
    row(
        column(
            unemp_plot,
        ),
        column(
            unemp_window_size_slider,
            unemp_date_range_slider,
            styles=Styles(
                padding='0 30px 30px',
                margin_top='auto',
                margin_bottom='auto',
            ),
        ),
        styles=Styles(
            padding='0 30px 30px',
            border_left="1px solid #ccc",
            margin_left="80px",
            border_right="1px solid #ccc",
            border_bottom="1px solid #ccc",
            margin_right="80px",
            margin_bottom="30px",
            border_bottom_left_radius="30px",
            border_bottom_right_radius="30px",
            # background_color="#f9f9f9",
        ),
    ),
    row(
        column(
            Div(
                text="""
                    <h2>Takeaways...</h2>
                    <p>Perhaps you're also skeptical of the usefulness of the Phillips Curve. Do
                    you think one of the other two relationships make more sense? It's probably
                    easy to agree that compensation might be inversely correlated with employment
                    (or inversely correlated with <b>un</b>employment).</p>
                    <p>More interestingly, what about the productivity vs. compensation relationship?
                    Do you think it shows a direct but leading/lagging relation, or do you think
                    it's inverse or otherwise not correlated? Or do you know of an entirely different
                    phenomenon?</p>
                    <p>There are many still undiscovered...</p>
                    """,
                width=680,
                stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
            ),
            width=650,
            styles=Styles(
                margin='30px 60px 30px 75px',
            ),
        ),
        column(
            Div(
                text="""
                    <h4>Footnotes</h4>
                    <p>
                        <sup>1</sup> <a href="http://www2.harpercollege.edu/mhealy/eco212i/lectures/ch12-17.htm" target=_blank">http://www2.harpercollege.edu/mhealy/eco212i/lectures/ch12-17.htm</a><br>
                        <sup>1</sup> <a href="https://www.nytimes.com/2024/04/12/opinion/inflation-economics-economists.html" target=_blank">https://www.nytimes.com/2024/04/12/opinion/inflation-economics-economists.html</a><br>
                        <sup>3</sup> <a href="https://en.wikipedia.org/wiki/Phillips_curve" target="_blank">https://en.wikipedia.org/wiki/Phillips_curve</a>"<br>
                        <sup>4</sup> <a href="https://www.bls.gov/opub/btn/volume-6/pdf/understanding-the-labor-productivity-and-compensation-gap.pdf" target=_blank">https://www.bls.gov/opub/btn/volume-6/pdf/understanding-the-labor-productivity-and-compensation-gap.pdf</a><br>
                        <sup>5</sup> <a href="https://www.imf.org/en/News/Articles/2023/05/15/sp-role-inflation-expectations-monetary-policy-tobias-adrian" target=_blank">https://www.imf.org/en/News/Articles/2023/05/15/sp-role-inflation-expectations-monetary-policy-tobias-adrian</a><br>
                        <sup>6</sup> <a href="https://machinelearningmastery.com/curve-fitting-with-python/" target=_"blank">https://machinelearningmastery.com/curve-fitting-with-python/</a><br>
                        <sup>7</sup> <a href="https://www.researchgate.net/publication/264816027_Phillips_curve_inflation_and_unemployment_an_empirical_research_for_Greece#pf3" target=_"blank">https://www.researchgate.net/publication/264816027_Phillips_curve_inflation_and_unemployment_an_empirical_research_for_Greece#pf3</a>
                    </p>
                    <h4>Bureau of Labor Statistics</h4>
                    <ul>
                        <li>Employment Cost Index <a href="https://www.bls.gov/eci/home.htm" target="_blank">Website</a> | <a href="https://www.bls.gov/eci/data.htm" target="_blank">Data</a></li>
                        <li>Consumer Price Index <a href="https://www.bls.gov/cpi/" target="_blank">Website</a> | <a href="https://www.bls.gov/cpi/data.htm" target="_blank">Data</a></li>
                        <li>Current Population Survey <a href="https://www.bls.gov/cps/" target="_blank">Website</a> | <a href="https://www.bls.gov/cps/data.htm" target="_blank">Data</a></li>
                        <li> Productivity Survey <a href="https://www.bls.gov/productivity/" target="_blank">Website</a> | <a href="https://www.bls.gov/productivity/data.htm" target="_blank">Data</a></li>
                    </ul>
                    """,
                stylesheets=[InlineStyleSheet(css="h4 { margin-bottom: 0; }  p { font-size: 13px; margin: 1em 0; } ul { margin-top: 1em; padding-left: 1em; } ")],
            ),
            styles=Styles(
                margin='30px 60px 30px 75px',
            ),
            width=400,
        ),
    ),
)

# Stretch both.
cpi_unempl_gridplot = gridplot([[cpi_unempl_layout]])

# Save layout.
output_file("index.html")
save(cpi_unempl_gridplot)

'/Users/nketchum/Library/CloudStorage/Dropbox/Academics/SI 649/Narrative Viz/index.html'